In [10]:
import os
from dotenv import load_dotenv # 有更改就执行下这两行，可以把新的环境配置加载进来
from langchain_community.utilities.tavily_search import TAVILY_API_URL

load_dotenv()

TAVILY_API_KEY=os.getenv("TAVILY_API_KEY")

In [8]:
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()

In [11]:
from typing import Annotated

from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.messages import BaseMessage
from typing_extensions import TypedDict

from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition


class State(TypedDict):
    messages: Annotated[list, add_messages]


graph_builder = StateGraph(State)


tool = TavilySearchResults(max_results=2)
tools = [tool]
llm = ChatOpenAI(model="gpt-4o-mini")
llm_with_tools = llm.bind_tools(tools)


def chatbot(state: State):
    return {"messages": [llm_with_tools.invoke(state["messages"])]}


graph_builder.add_node("chatbot", chatbot)

tool_node = ToolNode(tools=[tool])
graph_builder.add_node("tools", tool_node)

graph_builder.add_conditional_edges(
    "chatbot",
    tools_condition,
)
# Any time a tool is called, we return to the chatbot to decide the next step
graph_builder.add_edge("tools", "chatbot")
graph_builder.add_edge(START, "chatbot")

In [13]:
graph = graph_builder.compile(checkpointer=memory)

In [21]:
config = {"configurable": {"thread_id": "2"}}

In [18]:
user_input = "Hi there! My name is Will."

# The config is the **second positional argument** to stream() or invoke()!
events = graph.stream(
    {"messages": [{"role": "user", "content": user_input}]},
    config,
    stream_mode="values",
)
for event in events:
    event["messages"][-1].pretty_print()

================================ Human Message =================================

Hi there! My name is Will.
================================== Ai Message ==================================

Hello Will! How can I assist you today?


In [22]:
user_input = "Remember my name?"

# The config is the **second positional argument** to stream() or invoke()!
events = graph.stream(
    {"messages": [{"role": "user", "content": user_input}]},
    config,
    stream_mode="values",
)
for event in events:
    event["messages"][-1].pretty_print()

================================ Human Message =================================

Remember my name?
================================== Ai Message ==================================

I don’t have memory in this chat, so I can’t remember past interactions or names. However, if you tell me your name, I can use it in our conversation!


In [27]:
from typing import TypedDict

class User(TypedDict):
    name: str
    age: int

# 创建一个符合 User 结构的字典
user1: User = {"name": "Alice", "age": 25}  # ✅ 正确

# 下面这些都会被类型检查器警告：
user2: User = {"name": "Alice"}  
user3: User = {"name": "Alice", "age": "25"}  # ❌ "age" 不是 int
user4: User = {"name": "Alice", "age": 25, "gender": "female"}  # ❌ 额外的键


In [ ]:
print(int(1))